<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/CSV_%ED%8C%8C%EC%9D%BC_%EC%B9%BC%EB%9F%BC_%EC%A7%91%EA%B3%84_%EC%8A%A4%ED%81%AC%EB%A6%BD%ED%8A%B8_(%EC%A4%91%EB%B3%B5_%EC%B2%98%EB%A6%AC_%EA%B0%95%ED%99%94).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import re
from collections import defaultdict

def process_csv_files_with_aggregation(input_folder, output_folder):
    """
    지정된 폴더의 CSV 파일들을 읽어 특정 패턴의 칼럼들의 평균 또는 합계를 계산하고,
    결과를 새로운 CSV 파일로 저장합니다.
    '전력'이 포함된 칼럼 그룹은 합계를 계산하고 _SUM 접미사를 붙입니다.
    숫자 접미사 패턴(_X, _X_Y)을 유연하게 처리합니다.
    기존에 _AVG 또는 _SUM으로 끝나는 집계된 열이 있으면 해당 열을 사용하고,
    관련 원본 열들의 재집계를 건너<0xEB><0><0xB5>니다.

    Args:
        input_folder (str): 입력 CSV 파일들이 있는 폴더 경로.
        output_folder (str): 처리된 CSV 파일들을 저장할 폴더 경로.
    """
    # 출력 폴더가 존재하지 않으면 생성합니다.
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"출력 폴더 '{output_folder}'를 생성했습니다.")

    # 집계 대상 칼럼을 식별하기 위한 정규 표현식입니다. (예: data_1, feature_4_10)
    column_pattern = re.compile(r"^(.*?)_(\d+)(_\d+)?$")

    # 입력 폴더 내의 모든 파일을 확인합니다.
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            input_file_path = os.path.join(input_folder, filename)
            output_file_name = f"{os.path.splitext(filename)[0]}_PROCESSED.csv"
            output_file_path = os.path.join(output_folder, output_file_name)

            print(f"파일 '{filename}' 처리 중...")

            try:
                df = pd.read_csv(input_file_path)

                if df.empty:
                    print(f"  파일 '{filename}'이 비어있습니다. 원본 내용 그대로 '{output_file_name}'으로 저장합니다.")
                    df.to_csv(output_file_path, index=False, encoding='utf-8-sig')
                    continue

                cols_to_aggregate_map = defaultdict(list)
                original_cols_to_keep = []
                # 이미 _AVG 또는 _SUM으로 끝나는 집계된 열의 기본 이름을 저장하는 세트
                found_aggregated_bases = set()

                # 1. 칼럼 분류: 기존 집계 열, 신규 집계 대상 원본 열, 기타 유지 열
                for col in df.columns:
                    is_col_handled_as_existing_aggregate = False
                    # 칼럼이 이미 _AVG 또는 _SUM으로 끝나는지 확인
                    if col.endswith('_AVG') or col.endswith('_SUM'):
                        # 기본 이름 추출 (예: "feature_AVG" -> "feature")
                        base_of_existing_aggregate = col.rsplit('_', 1)[0]
                        found_aggregated_bases.add(base_of_existing_aggregate)
                        original_cols_to_keep.append(col) # 기존 집계 열은 그대로 유지
                        is_col_handled_as_existing_aggregate = True

                    # 기존 집계 열이 아니라면, 패턴 매칭 시도
                    if not is_col_handled_as_existing_aggregate:
                        match = column_pattern.match(col)
                        if match:
                            # 집계 대상이 될 수 있는 원본 열 (예: data_1)
                            base_name = match.group(1)
                            cols_to_aggregate_map[base_name].append(col)
                        else:
                            # 기타 칼럼 (예: Timestamp, Notes 등)
                            original_cols_to_keep.append(col)

                # 최종 결과 데이터프레임 파트 초기화 (유지할 원본 칼럼들)
                # df[original_cols_to_keep]가 비어있을 수도 있으므로 .copy() 주의
                if original_cols_to_keep:
                    result_df_parts = [df[original_cols_to_keep].copy()]
                else:
                    result_df_parts = []


                # 2. 신규 집계 처리
                if not cols_to_aggregate_map and not original_cols_to_keep : # 모든 칼럼이 비었을 경우
                     print(f"  파일 '{filename}'에 처리할 칼럼이 없습니다.")
                elif not cols_to_aggregate_map:
                    print(f"  파일 '{filename}'에서 신규 집계 대상 칼럼을 찾지 못했습니다. 기존 집계/원본 칼럼만 유지됩니다.")

                for base_name, related_cols in cols_to_aggregate_map.items():
                    # 만약 이 base_name에 해당하는 기존 _AVG/_SUM 열이 이미 존재한다면, 신규 집계 건너뛰기
                    if base_name in found_aggregated_bases:
                        print(f"  '{base_name}' 그룹에 대한 집계 건너뛰기: 이미 '{base_name}_AVG' 또는 '{base_name}_SUM'과 같은 집계된 열이 존재합니다.")
                        continue

                    if related_cols:
                        numeric_cols_df = df[related_cols].apply(pd.to_numeric, errors='coerce')

                        agg_col_name = ""
                        agg_series = None

                        if "전력" in base_name:
                            agg_col_name = f"{base_name}_SUM"
                            agg_series = numeric_cols_df.sum(axis=1, skipna=True)
                            print(f"  칼럼 그룹 '{base_name}' (칼럼: {related_cols})의 합계를 '{agg_col_name}'으로 계산했습니다.")
                        else:
                            agg_col_name = f"{base_name}_AVG"
                            agg_series = numeric_cols_df.mean(axis=1, skipna=True)
                            print(f"  칼럼 그룹 '{base_name}' (칼럼: {related_cols})의 평균을 '{agg_col_name}'으로 계산했습니다.")

                        if agg_series is not None:
                            agg_series.name = agg_col_name
                            result_df_parts.append(agg_series)

                # 3. 최종 데이터프레임 생성
                if not result_df_parts :
                    final_df = pd.DataFrame() # 모든 파트가 비어있으면 빈 DF 생성
                else:
                    # 비어있지 않은 Series/DataFrame만 concat 대상으로 필터링
                    valid_parts = [part for part in result_df_parts if not (isinstance(part, (pd.DataFrame, pd.Series)) and part.empty)]
                    if not valid_parts: # 유효한 파트가 하나도 없으면
                         final_df = pd.DataFrame()
                    else:
                        final_df = pd.concat(valid_parts, axis=1)

                final_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')
                print(f"  파일 '{filename}' 처리 완료. 저장 위치: '{output_file_path}'")

            except pd.errors.EmptyDataError:
                print(f"  파일 '{filename}'이 비어있거나 유효한 CSV 형식이 아닙니다. 건너<0xEB><0><0xB5>니다.") # "건너<0xEB><0><0xB5>니다" -> "건너뜁니다"
            except Exception as e:
                print(f"  파일 '{filename}' 처리 중 오류 발생: {e}")
    print("\n모든 CSV 파일 처리가 완료되었습니다.")

if __name__ == '__main__':
    input_directory = "your_input_csv_folder_path"
    output_directory = "your_output_csv_folder_path"

    if input_directory == "your_input_csv_folder_path" or \
       output_directory == "your_output_csv_folder_path":
        print("스크립트 하단의 'input_directory'와 'output_directory' 변수에\n"
              "실제 폴더 경로를 입력한 후 다시 실행해주세요.")
    else:
        process_csv_files_with_aggregation(input_directory, output_directory)